In [1]:
import json
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np

In [2]:
def load(i):
    start = i*1000
    path = "data/mpd.slice." + str(start) + "-" + str(start+999) + ".json"
    data = json.load(open(path,'r'))
    return pd.DataFrame.from_dict(data['playlists'], orient='columns')

In [3]:
# get tracks from playlists 

songPlaylistArray = []
start = 0
while start != 1:
     thisSlice = load(start)
     songPlaylistArray.append(thisSlice)
     start+= 1
train = pd.concat(songPlaylistArray)

In [4]:
train

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37,NaN
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21,NaN
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31,NaN
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86,NaN
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
995,old,false,995,1507852800,41,40,1,"[{'pos': 0, 'artist_name': 'Katrina', 'track_u...",8,9917901,36,NaN
996,Daze,false,996,1479254400,17,17,1,"[{'pos': 0, 'artist_name': 'PARTYNEXTDOOR', 't...",13,3699248,15,NaN
997,rap,false,997,1410307200,119,98,1,"[{'pos': 0, 'artist_name': 'LoveRance', 'track...",63,27538723,82,NaN
998,Country,false,998,1507939200,108,75,1,"[{'pos': 0, 'artist_name': 'Hunter Hayes', 'tr...",37,24950143,40,NaN


In [5]:
train.loc[train['pid'] == 844]

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
844,nostalgia,false,844,1487289600,150,86,1,"[{'pos': 0, 'artist_name': 'The Strokes', 'tra...",12,33568346,55,NaN


In [6]:
# get table where each row represents a song within a playlist

exploded_train = train.explode('tracks')
exploded_train.reset_index(drop=True, inplace=True)
normalized_tracks = pd.json_normalize(exploded_train['tracks'])
train_playlist_songs = pd.concat([exploded_train.drop(columns='tracks'), normalized_tracks], axis=1)

In [7]:
train_playlist_songs

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,num_edits,duration_ms,num_artists,description,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67498,thinking of you,false,999,1507766400,44,43,1,34,9065801,37,NaN,39,James Arthur,spotify:track:5uCax9HTNlzGybIStD3vDh,spotify:artist:4IWBUUAFIplrNtaOHcJPRM,Say You Won't Let Go,spotify:album:7oiJYvEJHsmYtrgviAVIBD,211466,Back from the Edge
67499,thinking of you,false,999,1507766400,44,43,1,34,9065801,37,NaN,40,Big Words,spotify:track:0P1oO2gREMYUCoOkzYAyFu,spotify:artist:0sHN89qak07mnug3LVVjzP,The Answer,spotify:album:5jrsRHRAmetu5e7RRBoxj7,263679,"Hollywood, a Beautiful Coincidence"
67500,thinking of you,false,999,1507766400,44,43,1,34,9065801,37,NaN,41,Allan Rayman,spotify:track:2oM4BuruDnEvk59IvIXCwn,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,25.22,spotify:album:3CbNgBzI7r9o0F6VjH9sTY,189213,Roadhouse 01
67501,thinking of you,false,999,1507766400,44,43,1,34,9065801,37,NaN,42,Jon Jason,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,spotify:artist:77bNdkKYBBmc30CisCA6tE,Good Feeling,spotify:album:2dZ7oVNQBeLlpoUYfbEsJP,194720,Good Feeling


In [8]:
#train_playlist_songs.columns.tolist()

In [9]:
# Discovering of data
#train_playlist_songs.isnull().sum()

### Take some playlist as user's preferencies / TODO Temporary solution 

In [10]:
# Take un example for user's playlist
def load2(i):
    start = i*10000
    path = "data/mpd.slice." + str(start) + "-" + str(start+999) + ".json"
    data = json.load(open(path,'r'))
    return pd.DataFrame.from_dict(data['playlists'], orient='columns')

user_PL = []
user_PL.append(load2(1))
df_user_PL = pd.concat(user_PL)
df_user_PL = df_user_PL.iloc[1]
#df_user_PL

#### Select all playlists according track_uri from user's playlist

In [15]:
# URI's of user's prefered tracks as list

pref_tracksURI = [el['track_uri'] for el in df_user_PL['tracks']]

# Remove doublons
pref_tracksURI_list = list(set(pref_tracksURI))

print("nb of prefered tracks = ", len(pref_tracksURI_list))
pref_tracksURI_list

nb of prefered tracks =  27


['spotify:track:2DwFMLTHlhiQHBC3XLiJYr',
 'spotify:track:4BodkdQk2Ouo6BFbLqoWtG',
 'spotify:track:37LNThlFJaNmi9SeLrv8H6',
 'spotify:track:5Ozo0lsCaVNJPlKC7RtDnp',
 'spotify:track:23oxJmDc1V9uLUSmN2LIvx',
 'spotify:track:714Lw0m2SmCEhKSPw0Dn8J',
 'spotify:track:48O6kz322Dzu1R6Al5147q',
 'spotify:track:0gdLTqxAY4DDUQxXzmwj1z',
 'spotify:track:5lDriBxJd22IhOH9zTcFrV',
 'spotify:track:1FMHNVeJ9s1x1l1WlaRs2I',
 'spotify:track:7I1P7Dam9l2fhMRMkshH16',
 'spotify:track:7oK9VyNzrYvRFo7nQEYkWN',
 'spotify:track:3LHMFgMolEbhhGBC47OuAi',
 'spotify:track:5fGkcxOENUG2TX16Ij26oz',
 'spotify:track:5vi81Z4ZOO5szbys3uHLUA',
 'spotify:track:7GCaZax7ExKSNYFv8eQCvL',
 'spotify:track:2l57cfmCnOkwNX1tky02n1',
 'spotify:track:3iL2l5gUqyPS6vDwJFgJTR',
 'spotify:track:4dYeLvp1VhtMRBQvSUScoz',
 'spotify:track:6TlRNJaezOdzdECnQeRuMM',
 'spotify:track:4jVhSbMMHctghoOTFuLPB2',
 'spotify:track:2Eh5ro0vWcZQY7r1YeYqxc',
 'spotify:track:2WOiei9edVwlABd8cJfzja',
 'spotify:track:26QSvu9Eu82RKCgPfuZqlk',
 'spotify:track:

# User-based recommendation (based on playlist)

In [16]:
# Select playlists contained user's prefered tracks
pref_in_train = train_playlist_songs.loc[train_playlist_songs['track_uri'].isin(pref_tracksURI_list)]
pref_in_train

# Range them according to relevance
most_relevant = pref_in_train['pid'].value_counts().rename_axis('pid').reset_index(name='Frequency')

print("inique values:\n", most_relevant.nunique())

most_relevant

inique values:
 pid          63
Frequency     5
dtype: int64


,pid,Frequency
0,844,5
1,987,4
2,45,4
3,921,3
4,895,3
...,...,...
58,565,1
59,601,1
60,625,1
61,642,1


In [17]:
# Range by most frequent songs
most_freq_songs = pd.DataFrame(train_playlist_songs.groupby(['track_uri'])['pid'].count()).reset_index().rename(columns= {'pid':'song_freq'}).sort_values(by=['song_freq'], ascending=False)

most_freq_songs

,track_uri,song_freq
8631,spotify:track:1xznGGDReH1oQq0xzbwXa3,55
32396,spotify:track:7KXjTSCq5nL1LoYtL7XAwS,52
34365,spotify:track:7yyRTcZmCiyzzJlNzGC9Ol,50
31775,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,46
15675,spotify:track:3a1lNhkSLSkpJE4MSHpDu9,44
...,...,...
13397,spotify:track:346k4djpLH5R6JQgiW0zSQ,1
13396,spotify:track:346GCw2msr77HA5I5NFsKS,1
13392,spotify:track:340k3JIT2OD06pNNpH38KW,1
13391,spotify:track:340eA3P5yCo3QYeOCG1lYF,1


## User-based recommendation RESULT

In [20]:
# Make list of playlists in range of relevance
pids = most_relevant['pid'].tolist()

# Make list of songs in range of frequency
uris = most_freq_songs['track_uri'].tolist()

# Exclude preferencies from proposition
sans_preferences = train_playlist_songs[train_playlist_songs.pid.isin(pids) & ~train_playlist_songs.track_uri.isin(pref_tracksURI_list)][['pid', 'track_uri']]

# Add playlist frequency info
new_one = sans_preferences.merge(most_relevant, left_on='pid', right_on='pid').rename(columns= {'Frequency':'playlist_freq'})

# Add track frequency info, sort by playlist_freq first then by song_freq
new_one_1 = new_one.merge(most_freq_songs, left_on='track_uri', right_on='track_uri').sort_values(by=['playlist_freq', 'song_freq'], ascending=False)

# Exclude doublons from proposition
sans_doublons = new_one_1[['track_uri', 'playlist_freq', 'song_freq']].drop_duplicates(['track_uri'])

# First 500 tracks
recommended = sans_doublons[:500]['track_uri'].tolist()
recommended

['spotify:track:5XJJdNPkwmbUwE79gv0NxK',
 'spotify:track:6C7RJEIUDqKkJRZVWdkfkH',
 'spotify:track:0WqIKmW4BTrj3eJFmnCKMv',
 'spotify:track:6RcQOut9fWL6FSqeIr5M1r',
 'spotify:track:7w87IxuO7BDcJ3YUqCyMTT',
 'spotify:track:4bPQs0PHn4xbipzdPfn6du',
 'spotify:track:27L8sESb3KR79asDUBu8nW',
 'spotify:track:1TfqLAPs4K3s2rJMoCokcS',
 'spotify:track:6I9VzXrHxO9rA9A5euc8Ak',
 'spotify:track:2gam98EZKrF9XuOkU13ApN',
 'spotify:track:2QjOHCTQ1Jl3zawyYOpxh6',
 'spotify:track:1fLdeDTrJWNkwOeFyAVLvF',
 'spotify:track:7uKcScNXuO3MWw6LowBjW1',
 'spotify:track:5oQcOu1omDykbIPSdSQQNJ',
 'spotify:track:6L89mwZXSOwYl76YXfX13s',
 'spotify:track:6ndmKwWqMozN2tcZqzCX4K',
 'spotify:track:4sebUbjqbcgDSwG6PbSGI0',
 'spotify:track:18GC3F8YPGB8CePXcTUizQ',
 'spotify:track:3HooZZPp0evFShqaJ2Pwer',
 'spotify:track:1jNOi6m3Hn8nLEeHCp5Msr',
 'spotify:track:30VrBsh1STRBoIrhQOAwzK',
 'spotify:track:6GIrIt2M39wEGwjCQjGChX',
 'spotify:track:557un1HgwYMuqfWGSTmnxw',
 'spotify:track:2TfSHkHiFO4gRztVIkggkE',
 'spotify:track:

# Item-based recommendation (based on tracks)